# Weekly Insight Packet — 2025-10-16

_Shell notebook for charts, narrative tables, and analyzer joins. Populate once Supabase credentials and fresh NDJSON export land._

## Setup

- Configure Supabase + GA MCP environment variables before running.
- Ensure `artifacts/logs/` has the latest NDJSON export from reliability.
- Install required packages via `npm install` / `pip install -r notebooks/requirements.txt` if applicable.

In [ ]:
# Bootstrap shared config once credentials arrive.
import json
import math
from pathlib import Path

SUPABASE_EXPORT_PATH = Path("artifacts/logs/supabase_decision_export_2025-10-10T07-29-39Z.ndjson")

if not SUPABASE_EXPORT_PATH.exists():
    raise FileNotFoundError(f"Expected NDJSON export missing: {SUPABASE_EXPORT_PATH}")


In [ ]:
# Load NDJSON export into memory for quick summaries.
with SUPABASE_EXPORT_PATH.open("r", encoding="utf-8") as ndjson_file:
    decision_records = [json.loads(line) for line in ndjson_file if line.strip()]

if not decision_records:
    raise ValueError(f"NDJSON export {SUPABASE_EXPORT_PATH} is empty")

total_records = len(decision_records)
success_count = sum(1 for row in decision_records if (row.get("status") or "").upper() == "SUCCESS")
failure_count = sum(1 for row in decision_records if (row.get("status") or "").upper() not in ("SUCCESS", ""))
timeout_ids = [row.get("decisionId") for row in decision_records if (row.get("status") or "").upper() == "TIMEOUT"]
durations = [float(row.get("durationMs", 0.0)) for row in decision_records if row.get("durationMs") is not None]
if durations:
    durations_sorted = sorted(durations)
    average_duration_ms = sum(durations_sorted) / len(durations_sorted)
    p95_index = max(0, min(len(durations_sorted) - 1, math.ceil(0.95 * len(durations_sorted)) - 1))
    p95_duration_ms = durations_sorted[p95_index]
else:
    average_duration_ms = 0.0
    p95_duration_ms = 0.0

retry_distribution = {}
for row in decision_records:
    attempts = int(row.get("attempt", 1) or 1)
    retry_distribution[attempts] = retry_distribution.get(attempts, 0) + 1
retry_distribution = dict(sorted(retry_distribution.items()))

failure_rate_pct = round((failure_count / total_records) * 100, 2) if total_records else 0.0

decision_sync_snapshot = {
    "total": total_records,
    "success": success_count,
    "failure": failure_count,
    "failure_rate_pct": failure_rate_pct,
    "timeouts": timeout_ids,
    "avg_duration_ms": round(average_duration_ms, 2),
    "p95_duration_ms": round(p95_duration_ms, 2),
    "retry_distribution": retry_distribution,
}

decision_sync_snapshot


## KPI Trend Queries

_Placeholder: replace with actual warehouse query pulls once Supabase credentials are unlocked._

In [ ]:
# TODO: Fetch sales delta, SLA breach rate, traffic anomalies into data frames.
sales_delta_trend = None
sla_breach_trend = None
traffic_anomaly_trend = None


## Decision-Sync Analyzer Join

_Placeholder: invoke scripts/ops/analyze-supabase-logs.ts and embed summary artifacts._

In [ ]:
# Load analyzer summary for downstream visualizations.
ANALYZER_SUMMARY_PATH = Path("artifacts/monitoring/supabase-sync-summary-latest.json")

if ANALYZER_SUMMARY_PATH.exists():
    with open(ANALYZER_SUMMARY_PATH, "r", encoding="utf-8") as summary_file:
        analyzer_summary = json.load(summary_file)
else:
    analyzer_summary = {}


## Visualization Placeholders

- Insert matplotlib/plotly charts for KPI trends.
- Render analyzer pie chart + latency histograms.
- Summarize GA MCP readiness checklist progress.

In [ ]:
# Visualization helpers will be populated once additional telemetry arrives.
def render_kpi_trends():
    return None

def render_decision_sync_charts():
    return decision_sync_snapshot

def summarize_ga_mcp_readiness():
    return {
        "status": "blocked",
        "details": "Awaiting OCC-INF-221 staging credentials before GA MCP parity can run."
    }


## Narrative Draft

_Use this section to draft the executive summary text and key callouts aligned with manager packet outline._

In [ ]:
# Draft narrative populated with current telemetry snapshot.
weekly_narrative = {
    "headline": (
        f"Decision sync remains partially degraded with {decision_sync_snapshot['failure_rate_pct']}% failures across {decision_sync_snapshot['total']} records (timeouts: {decision_sync_snapshot['timeouts']})."
    ),
    "decision_sync": (
        f"Average latency sits at {decision_sync_snapshot['avg_duration_ms']} ms (p95 {decision_sync_snapshot['p95_duration_ms']} ms) with retry distribution {decision_sync_snapshot['retry_distribution']}."
    ),
    "ga_mcp": "GA MCP contract tests remain blocked pending OCC-INF-221 credential drop; parity rerun queued once secrets land.",
    "next_steps": [
        "Monitor hourly NDJSON exports and append expanded metrics/visuals.",
        "Coordinate with QA/design on modal asset capture once staging mock=0 stabilizes.",
        "Generate KPI trend queries after telemetry parity validates live data.",
    ],
}
weekly_narrative
